#Task #1: 資本資産価格モデル(CAPM)の理論と直感を理解する 

# task #2: ライブラリ/データセットのインポートとストックデータの可視化

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [3]:
# 株式データファイルの読み込み
stocks_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Python & ML in Finance/Part 2. Financial Analysis in Python/stock.csv')
stocks_df

FileNotFoundError: ignored

In [ ]:
# 日付を基準にデータをソート
stocks_df = stocks_df.sort_values(by = ['Date'])
stocks_df

In [ ]:
# 初値を基準にして価格を正規化する関数
def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

**ミニチャレンジ #1:**
- Plotly Expressを使って、株式データをインタラクティブにプロットする関数を作成する。
- **この関数を呼び出し、株式データを可視化する**。
- **関数を呼び出して、正規化された株式データを可視化する**。
- **Note: 行き詰ったら以前の講義をチェックしましょう**。


# task #3: 日次リターンの計算

In [ ]:
# 日次リターンを計算する関数 
def daily_return(df):

  df_daily_return = df.copy()
  
  # 各銘柄をループする
  for i in df.columns[1:]:
    
    # 銘柄に属する各行をループする
    for j in range(1, len(df)):
      
      # 前日からの変化率を計算する
      df_daily_return[i][j] = ((df[i][j]-df[i][j-1])/df[i][j-1]) * 100
    
    # 前の値が利用できないので, 最初の行の値をゼロにします。
    df_daily_return[i][0] = 0
  return df_daily_return

In [ ]:
# 日次リターンの取得
stocks_daily_return = daily_return(stocks_df)
stocks_daily_return

**ミニ・チャレンジ #2:**
- **次のうちどれがより高い平均日次リターンを持っていますか**？
   - S&P500またはGoogle
   - S&P500 or Amazon


# task #4: 単一銘柄のベータを計算する

In [ ]:
# 任意の銘柄を選択、例えばAppleとします 
stocks_daily_return['AAPL']

In [ ]:
# S&P500(市場)を選択
stocks_daily_return['sp500']

In [ ]:
# 選択した銘柄とS&P500(市場)の散布図をプロットする
stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = 'AAPL')

In [ ]:
# 選択された銘柄とS&P500の間に多項式をフィッティングします (Poly with order = 1 is a straight line)

# βは直線回帰線(市場リターン対株式リターン)の傾きを表す。
# ベータは、市場全体(S&P500)と比較した証券やポートフォリオのボラティリティーやシステマティック・リスクを示す指標です。
# ベータはCAPMで使用され、資産のシステマティック・リスクと期待リターンの関係を表す 

# Beta = 1.0, これは、その価格活動が市場と強く相関していることを示します。
# Beta < 1, これは、その証券が理論的に市場よりも変動が小さいことを示しています (Ex: Utility stock)。この銘柄が含まれている場合、その銘柄を含まない同じポートフォリオと比較して、ポートフォリオのリスクが低くなります。
# ベータ値が1を超えると、その銘柄の価格が市場よりも変動しやすいことを意味します。例えば、テスラのベータ値は1.26で、市場よりも26%変動が大きいことを示しています。
# ♪ハイテク株は一般的にS&P500よりもベータ値が高いが、超過リターンもある
# MGMはS&P500よりも65%ボラティリティが高い!



beta, alpha = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['AAPL'], 1)
print('Beta for {} stock is = {} and alpha is = {}'.format('AAPL', beta, alpha))  

In [ ]:
# では、散布図と直線を一枚の図にプロットしてみましょう。
stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = 'AAPL')

# αとβをパラメータとした直線の方程式 
# 直線の方程式は y = β * rm + α
plt.plot(stocks_daily_return['sp500'], beta * stocks_daily_return['sp500'] + alpha, '-', color = 'r')


**ミニチャレンジ#3:**
- **Tesla Inc.のベータを計算し、Appleと比較する**。


# task #5: CAPMの公式を個別銘柄に適用する 

In [ ]:
beta

In [ ]:
# S&P500の日次平均収益率を計算してみましょう
stocks_daily_return['sp500'].mean()

In [ ]:
# S&P500の年率収益率を計算してみましょう。
# 1年365日のうち、週末（土日）の104日間は証券取引所が閉鎖されていることに注意してください。
# https://dqydj.com/sp-500-return-calculator/ で答えを確認してください。

rm = stocks_daily_return['sp500'].mean() * 252
rm

In [ ]:
# リスクフリーレートをゼロと仮定する
# また、リスクフリーレートとして、10年物の米国国債の利回りを使うこともできる
rf = 0 

# CAPMを使って任意の証券(APPL)のリターンを計算する  
ER_AAPL = rf + ( β * (rm-rf) ) 

In [ ]:
ER_AAPL

**ミニ・チャレンジ #4:**
- **CAPMの公式を適用してAT&Tのリターンを計算する**。 


# task #6: すべての銘柄のベータを計算する

In [ ]:
# すべてのベータとアルファのプレースホルダーを作ろう（空の辞書）
ベータは {} です。
アルファ = {}

# すべての銘柄のデイリーリターンをループする
for i in stocks_daily_return.columns:

  # 日付とS&P500のカラムを無視する 
  if i != 'Date' and i != 'sp500':
    # 各個別銘柄とS&P500(市場)の散布図をプロットする
    stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = i)
    
    # 各銘柄とS&P500の間に多項式をフィッティング (Poly with order = 1 is a straight line)
    b, a = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return[i], 1)
    
    plt.plot(stocks_daily_return['sp500'], b * stocks_daily_return['sp500'] + a, '-', color = 'r')
    
    ベータ[i] = b
    
    アルファ[i] = a
    
    plt.show()

In [ ]:
# 各銘柄のベータを表示してみよう 
beta

In [ ]:
# 各銘柄のアルファを表示してみましょう
# アルファは市場（S&P500）に勝つための戦略の能力を表す
# アルファは "超過収益 "または "異常収益率 "を示す 
# テスラのアルファがプラス0.175ということは、ポートフォリオのリターンがベンチマークであるS&P500指数を17%上回ったことを意味します。
alpha


**ミニチャレンジ #5:**
- **Plotly Expressを使用して、S&P500の日次リターンと各銘柄を比較したインタラクティブなプロットを作成してください**。
- **ベストフィットの直線（アルファ値とベータ値を含む）をプロットする**。 


# task #7: capm formulaを適用してポートフォリオのリターンを計算する。

In [ ]:
# 全銘柄のリストを取得する
keys = list(beta.keys())
keys

In [ ]:
# 期待リターン辞書の定義
rf = 0 # ここではリスクフリーレートをゼロとする
rm = stocks_daily_return['sp500'].mean() * 252 # これは、市場の期待リターンです。
rm

In [ ]:
for i in keys:
 # CAPMを使って、すべての証券のリターンを計算  
  ER[i] = rf + ( beta[i] * (rm-rf) ) 

In [ ]:
for i in keys:
  print('Expected Return Based on CAPM for {} is {}%'.format(i, ER[i]))

In [ ]:
# ポートフォリオのウエイトが等しいと仮定
portfolio_weights = 1/8 * np.ones(8) 
portfolio_weights

In [ ]:
# ポートフォリオのリターンを計算 
ER_portfolio = sum(list(ER.values()) * portfolio_weights)
ER_portfolio

In [ ]:
print('Expected Return Based on CAPM for the portfolio is {}%\n'.format(ER_portfolio))

**ミニ・チャレンジ#6:**
- AppleとAmazonに50%ずつしか配分していないと仮定して，ポートフォリオの期待リターンを計算してください。


# **AWESOME JOB!**

# **ミニ・チャレンジの解決策**)

**ミニチャレンジ#1の解答:**
- Plotly Expressを使用して、株式データをインタラクティブにプロットする関数を作成する**。
- **この関数を呼び出して、株式データを可視化します**。
- **関数を呼び出して、正規化された株式データを可視化する**。
- **Note: 行き詰ったら以前の講義をチェックしましょう**。
# インタラクティブプロットを行う関数


In [ ]:
# インタラクティブプロットを行う関数
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
# インタラクティブチャートの描画
interactive_plot(stocks_df, 'Prices')

In [ ]:
# 正規化されたインタラクティブ・チャートを描画する
interactive_plot(normalize(stocks_df), 'Normalized Prices')

**ミニチャレンジ2解答：***。
- **次のうちどれがより高い平均日次収益率を持つか？
   - S&P500またはGoogle
   - S&P500 or Amazon


In [ ]:
stocks_daily_return.mean()
# S&P500の平均日次リターンは0.049%です。
# Amazonの平均日次収益率は0.15
# Google の平均日次リターンは 0.084% です。


**ミニチャレンジ #3 解決策： **
- # Tesla Inc.のベータを計算し、Appleと比較する**。


In [ ]:
# 選択した銘柄とS&P500の間に多項式をフィットさせる (Poly with order = 1 is a straight line)
beta, alpha = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['TSLA'], 1)
print(beta)  
# では、散布図と直線を一枚の図にプロットしてみましょう。
stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = 'TSLA')

# αとβのパラメータを持つ直線の方程式 
# 直線の方程式は y = β * rm + α
plt.plot(stocks_daily_return['sp500'], beta * stocks_daily_return['sp500'] + alpha, '-', color = 'r')


**ミニチャレンジ #4 解決策:**
- # CAPM式を適用してAT&Tのリターンを計算する**。 


In [ ]:
# まずAmazonのBetaを計算する必要があります
beta, alpha = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['T'], 1)
print('Beta for {} stock is = {} and alpha is = {}'.format('T', beta, alpha)) 

In [ ]:
# Calculate return for AT&T using CAPM  
ER_T = rf + ( beta * (rm - rf) ) 
print(ER_T)

**MINI CHALLENGE #5 SOLUTION:**
- **Using Plotly Express, plot an interactive plot showing S&P500 daily returns vs. every stock**
- **Plot the best fit straight line (with Alpha and Beta values)**  


In [ ]:
# Let's do the same plots but in an interactive way
# Explore some wierd points in the dataset: Tesla stock return was at 24% when the S&P500 return was -0.3%!

for i in stocks_daily_return.columns:
  
  if i != 'Date' and i != 'sp500':
    
    # Use plotly express to plot the scatter plot for every stock vs. the S&P500
    fig = px.scatter(stocks_daily_return, x = 'sp500', y = i, title = i)

    # Fit a straight line to the data and obtain beta and alpha
    b, a = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return[i], 1)
    
    # Plot the straight line 
    fig.add_scatter(x = stocks_daily_return['sp500'], y = b*stocks_daily_return['sp500'] + a)
    fig.show()

**MINI CHALLENGE #6 SOLUTION:**
- **Calculate the expected return for the portfolio assuming we only have 50% allocation in Apple and 50% in Amazon**


In [ ]:
ER['AMZN']

In [ ]:
# Calculate the portfolio return 
ER_portfolio = 0.5 * ER['AAPL'] +  0.5 * ER['AMZN']
ER_portfolio